[Fake News](https://www.kaggle.com/shahidkhan1/fake-news-detection-using-lstm-90-accuracy/data)

 **train.csv**: A full training dataset with the following attributes:


*   id: unique id for a news article
*  title: the title of a news article
* author: author of the news article
* text: the text of the article; could be incomplete
* label: a label that marks the article as potentially unreliable

>> 1: unreliable

>> 0: reliable

**test.csv**: A testing training dataset with all the same attributes at train.csv without the label.

**submit.csv**


In [1]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/ciclo2/BigData/proyecto-grupal"

In [2]:
!python -m pip install 'fsspec>=0.3.3' #

     |████████████████████████████████| 132 kB 11.5 MB/s 


In [10]:
!kaggle competitions download -c fake-news

 53% 5.00M/9.42M [00:00<00:00, 18.0MB/s]
100% 9.42M/9.42M [00:00<00:00, 31.4MB/s]
 67% 25.0M/37.0M [00:01<00:00, 22.5MB/s]
100% 37.0M/37.0M [00:01<00:00, 34.6MB/s]
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 42.7MB/s]


In [12]:
! unzip test.csv.zip
! unzip train.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


In [13]:
import pandas as pd
import psutil
import time

In [14]:
df_submit = pd.read_csv("/content/submit.csv")
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")


## Submit

In [5]:
df_submit.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [6]:
print("Cantidad de elementos SUBMIT:",len(df_submit.index))

Cantidad de elementos SUBMIT: 5200


## Train

In [9]:
df_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
print("Cantidad de elementos TRAIN:",len(df_train.index))

Cantidad de elementos TRAIN: 20800


In [11]:
display(type(df_train))
display((df_train.count(), len(df_train.columns)))

pandas.core.frame.DataFrame

(id        20800
 title     20242
 author    18843
 text      20761
 label     20800
 dtype: int64, 5)

In [12]:
df_train = df_train[df_train.text.notna()]

## Test

In [7]:
df_test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [14]:
print("Cantidad de elementos TEST:",len(df_test.index))

Cantidad de elementos TEST: 5200


In [15]:
display(type(df_test))
display((df_test.count(), len(df_test.columns)))

pandas.core.frame.DataFrame

(id        5200
 title     5078
 author    4697
 text      5193
 dtype: int64, 4)

In [16]:
df_test = df_test[df_test.text.notna()]


## DASK: lecturas

In [6]:
import dask.dataframe as dd #

### Submit


In [16]:
%%time
start = time.time()
ddf_submit = dd.from_pandas(df_submit, npartitions = 2) 
ddf_submit.compute()
#print(psutil.Process().memory_info().rss / (1024 * 1024))



CPU times: user 2.75 ms, sys: 69 µs, total: 2.82 ms
Wall time: 2.56 ms


### Train

In [17]:
%%time
ddf_train = dd.from_pandas(df_train, npartitions = 2)
ddf_train.compute()
#print(psutil.Process().memory_info().rss / (1024 * 1024))


CPU times: user 219 ms, sys: 83.9 ms, total: 303 ms
Wall time: 299 ms


In [18]:
len(ddf_train.index)



20800

### Test

In [19]:
%%time
ddf_test = dd.from_pandas(df_test, npartitions = 2)
ddf_test.compute()
#print(psutil.Process().memory_info().rss / (1024 * 1024))


CPU times: user 66.3 ms, sys: 95 µs, total: 66.4 ms
Wall time: 67.3 ms


## DASK: Union Test y Submit

In [11]:
ddf_submit.compute()


,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1
...,...,...
5195,25995,0
5196,25996,1
5197,25997,0
5198,25998,1


In [24]:
#type(ddf_submit)
#type(ddf_test_not_null)
#ddf_test_merge_submit = ddf_submit.merge(ddf_test, on="id")
%%time
ddf_test_merge_submit =  ddf_test.merge(ddf_submit, on="id")
print(psutil.Process().memory_info().rss / (1024 * 1024))


686.38671875
CPU times: user 26.3 ms, sys: 10 ms, total: 36.3 ms
Wall time: 51.9 ms


In [22]:
!pip install partd #

In [25]:
ddf_test_merge_submit.compute()

,id,title,author,text,label
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",0
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...,1
2,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,1
3,20811,“Working Class Hero” by John Brennon,Doug Diamond,"Source: CNBC, article by Robert Ferris Arctic ...",1
4,20812,The Rise of Mandatory Vaccinations Means the E...,Shaun Bradley,Written by Shaun Bradley Mandatory vaccinati...,0
...,...,...,...,...,...
2571,25991,E3 Hands-On with ’South Park: The Fractured bu...,Lucas Nolan,The title of South Park: The Fractured but Who...,1
2572,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...,1
2573,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...,0
2574,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...,1


## DASK: remover valores null de train y test

1.   Test



In [ ]:
#ddf_test_merge_submit['text'].notnull().compute()

In [26]:
%%time
ddf_test_not_null = ddf_test_merge_submit[ddf_test_merge_submit['text'].notnull()].compute() 
ddf_test_not_null
print(psutil.Process().memory_info().rss / (1024 * 1024))


689.9453125
CPU times: user 126 ms, sys: 10.4 ms, total: 136 ms
Wall time: 130 ms


In [16]:
 print("Datos originales:", len(ddf_test), len(ddf_test.columns))
 print("Datos ddf_test_not_null:", len(ddf_test_not_null), len(ddf_test_not_null.columns))

Datos originales: 5200 4
Datos ddf_test_not_null: 5193 5


In [68]:
ddf_test_not_null = ddf_test_not_null.reset_index().drop_duplicates(subset='index', keep='last').set_index('index').sort_index()


In [69]:
ddf_test_not_null.index.duplicated()

array([False, False, False, ..., False, False, False])

1.  Train

In [ ]:
#ddf_train['text'].notnull().compute()

In [27]:
%%time
ddf_train_not_null = ddf_train[ddf_train['text'].notnull()].compute() 
ddf_train_not_null
print("memoria:",psutil.Process().memory_info().rss / (1024 * 1024))

memoria: 689.96875
CPU times: user 10.8 ms, sys: 11.9 ms, total: 22.7 ms
Wall time: 22.5 ms


In [18]:
 print("Datos originales:", len(ddf_train), len(ddf_train.columns))
 print("Datos ddf_train_not_null:", len(ddf_train_not_null), len(ddf_train_not_null.columns))

Datos originales: 20800 5
Datos ddf_train_not_null: 20761 5


## DASK: pre procesamiento

In [52]:
ddf_test_not_null["text"]

0       PALO ALTO, Calif.  —   After years of scorning...
1       Russian warships ready to strike terrorists ne...
2       42 mins ago 1 Views 0 Comments 0 Likes 'For th...
3       Source: CNBC, article by Robert Ferris Arctic ...
4       Written by Shaun Bradley   Mandatory vaccinati...
                              ...                        
2571    The title of South Park: The Fractured but Who...
2572    WASHINGTON  —   Gov. John Kasich of Ohio on Tu...
2573    Good morning. (Want to get California Today by...
2574    « Previous - Next » 300 US Marines To Be Deplo...
2575    Perhaps you’ve seen the new TV series whose pi...
Name: text, Length: 5193, dtype: object

In [71]:
ddf_test_not_null["text"]
#ddf_test_not_null

index
0       Videos #NoDAPL: Native American Leaders Vow to...
1       If at first you don’t succeed, try a different...
2       Trump is USA's antique hero. Clinton will be n...
3       Sunday on NBC’s “Meet the Press,” House Minori...
4       You are here: Home / *Articles of the Bound* /...
                              ...                        
2619    Dr. Caleb Alexander knows how easily older peo...
2620    Earth To Ammosexuals: NRA Admits No One Is Com...
2621    Public concern about faulty automobile airbags...
2622    Pres. Trump on if “tapes” exist of his convers...
2623    Of all the dysfunctions that plague the world’...
Name: text, Length: 2624, dtype: object

In [28]:
ddf_train_not_null["text"]

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        Ever get the feeling your life circles the rou...
2        Why the Truth Might Get You Fired October 29, ...
3        Videos 15 Civilians Killed In Single US Airstr...
4        Print \nAn Iranian woman has been sentenced to...
                               ...                        
20795    Rapper T. I. unloaded on black celebrities who...
20796    When the Green Bay Packers lost to the Washing...
20797    The Macy’s of today grew from the union of sev...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799      David Swanson is an author, activist, journa...
Name: text, Length: 20761, dtype: object

In [41]:
pip install emoji

     |████████████████████████████████| 170 kB 12.9 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=3df9dca09a1945a03a291fe385c98ed449232488b036ddbfd0c7a1965c3d869c
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [45]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string

In [46]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [47]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'this', 'do', 'once', 'your', 'while', 'no', 'was', 'how', "wouldn't", "you've", "haven't", 'above', 'does', 'ours', 'below', 'a', 'only', 'don', "mustn't", 'after', 'had', 'shan', 'i', 'having', 'ma', 'weren', 'in', 'if', 'shouldn', 'these', "weren't", "it's", 'are', 'few', 're', 'd', 'you', 'up', 'nor', 'at', "shouldn't", 'when', 'same', 've', 'needn', 'on', 'himself', 'hasn', "you'll", 'an', 'where', 'yours', 'yourself', "mightn't", 'between', 'can', 'such', 'own', 'itself', 'both', 'him', 'those', 'so', 's', "couldn't", 'whom', 'what', 'to', 'down', 'hadn', 'here', 'during', 'our', 'of', 'through', 'is', 'each', 'them', 'did', "hasn't", "won't", 'most', "that'll", "you'd", 'his', 't', 'will', 'before', 'into', 'out', 'y', "aren't", 'myself', 'the', 'yourselves', 'or', 'they', "needn't", 'mustn', 'wouldn', 'under', 'just', 'from', 'has', 'very', 'more', 'further', "hadn't", 'my', 'ourselves', 'that', 'than', 'who', 'wasn', "isn't", 'o', 'doesn', "wasn't", 'but', 'be', 'themselves',

In [48]:
import emoji
def give_emoji_free_text(text): 
    return emoji.get_emoji_regexp().sub(r'', text)

In [49]:
def preprocess_new_text(tweet):
  emoticon_string = r"""
  (?:
    [<>]?
    [:;=8]                     
    [\-o\*\']?                 
    [\)\]\(\[dDpP/\:\}\{@\|\\]       
    |
    [\)\]\(\[dDpP/\:\}\{@\|\\]
    [\-o\*\']?                 
    [:;=8]                     
    [<>]?
  )"""

  replacements = (
    ("á", "a"),
    ("é", "e"),
    ("í", "i"),
    ("ó", "o"),
    ("ú", "u"),
  )

  tweet = str(tweet)
  tweet = tweet.lower()
  # Remove urls
  tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
  # Remove user @ references and '#' from tweet
  tweet = re.sub(r'\@\w+|\#','', tweet)
  # Remove user ¿ and ¡ from tweet
  tweet = re.sub(r'\¿|\¡|\?|\!','', tweet)
  # Remove punctuations
  tweet = tweet.translate(str.maketrans('', '', string.punctuation))
  # remove graphical emoji
  tweet = give_emoji_free_text(tweet) 
  # remove textual emoji
  tweet = re.sub(emoticon_string,'',tweet)
  #remove accent mark
  for a, b in replacements:
    tweet = tweet.replace(a, b)

  # Remove stopwords
  tweet_tokens = word_tokenize(tweet)
  filtered_words = [w for w in tweet_tokens if not w in stop_words]
    
  return " ".join(filtered_words)

In [50]:
%%time
df_train.text = ddf_train_not_null["text"].apply(preprocess_new_text)
print(psutil.Process().memory_info().rss / (1024 * 1024))

840.6484375
CPU times: user 15min 29s, sys: 2.34 s, total: 15min 32s
Wall time: 15min 31s


In [76]:
df_train["text"]

0        house dem aide ’ even see comey ’ letter jason...
1        ever get feeling life circles roundabout rathe...
2        truth might get fired october 29 2016 tension ...
3        videos 15 civilians killed single us airstrike...
4        print iranian woman sentenced six years prison...
                               ...                        
20795    rapper unloaded black celebrities met donald t...
20796    green bay packers lost washington redskins wee...
20797    macy ’ today grew union several great names am...
20798    nato russia hold parallel exercises balkans 11...
20799    david swanson author activist journalist radio...
Name: text, Length: 20800, dtype: object

In [70]:
%%time
df_test.text = ddf_test_not_null["text"].apply(preprocess_new_text)
print(psutil.Process().memory_info().rss / (1024 * 1024))


979.6484375
CPU times: user 1min 58s, sys: 347 ms, total: 1min 59s
Wall time: 1min 59s


## DASK: modelamiento

In [82]:
import numpy as np

In [92]:
%%time
from nltk.stem import PorterStemmer
import re
import tensorflow
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
embedding_vector_features = 40
voc_size = 5000
sent_length = 20
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
print("memoria:",psutil.Process().memory_info().rss / (1024 * 1024))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None
memoria: 1686.9296875
CPU times: user 252 ms, sys: 5.97 ms, total: 258 ms
Wall time: 260 ms


In [79]:
voc_size = 5000
df_train_str = df_train["text"].astype(str)

onehot_repre = [one_hot(words, voc_size) for words in df_train_str]
onehot_repre[0]

[543,
 815,
 4721,
 3283,
 4968,
 3784,
 3252,
 3283,
 2563,
 3435,
 530,
 4643,
 4606,
 3034,
 2864,
 4203,
 1334,
 1891,
 3435,
 530,
 853,
 3385,
 2959,
 3901,
 2312,
 1348,
 4511,
 1092,
 686,
 1223,
 4528,
 2587,
 4594,
 718,
 4214,
 2555,
 3806,
 1522,
 2127,
 2308,
 462,
 2815,
 3252,
 673,
 543,
 638,
 4721,
 1218,
 322,
 613,
 1137,
 4322,
 1522,
 2803,
 1802,
 3252,
 2587,
 2520,
 2563,
 4155,
 4121,
 4028,
 2393,
 2499,
 2444,
 1479,
 3312,
 3283,
 4521,
 84,
 591,
 288,
 3497,
 1496,
 3283,
 415,
 3252,
 556,
 3015,
 57,
 1139,
 970,
 1594,
 841,
 1137,
 3252,
 3309,
 970,
 841,
 638,
 591,
 3183,
 543,
 1400,
 972,
 4754,
 1496,
 1576,
 4900,
 2393,
 770,
 2138,
 2702,
 3784,
 2584,
 2834,
 1387,
 2904,
 2563,
 537,
 4754,
 1594,
 1315,
 3435,
 530,
 4141,
 2719,
 2127,
 12,
 57,
 4121,
 4939,
 3846,
 4121,
 4013,
 4915,
 2393,
 1967,
 4356,
 1773,
 4061,
 130,
 4757,
 3435,
 530,
 2864,
 2075,
 1334,
 41,
 1137,
 4061,
 3252,
 2032,
 1791,
 4900,
 2393,
 63,
 3622,
 1066,

In [80]:
embedded_docs = pad_sequences(onehot_repre, padding='pre', maxlen=sent_length)
print(embedded_docs[0])

[4481 4701 4714 2499 1137  175 2142 2967  815 3188 3065 1859 4910 1191
  624 3524 4606 1831 4580 4910]


In [83]:
Y = df_train.label  
x_final = np.array(embedded_docs)
y_final = np.array(Y)

In [84]:
x_final.shape, y_final.shape

((20800, 20), (20800,))

In [85]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [93]:
%%time
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)
print("memoria:",psutil.Process().memory_info().rss / (1024 * 1024))

Epoch 1/10
218/218 [==============================] - 7s 25ms/step - loss: 0.4070 - accuracy: 0.8065 - val_loss: 0.3072 - val_accuracy: 0.8612
Epoch 2/10
218/218 [==============================] - 5s 24ms/step - loss: 0.2232 - accuracy: 0.9076 - val_loss: 0.2964 - val_accuracy: 0.8728
Epoch 3/10
218/218 [==============================] - 5s 23ms/step - loss: 0.1519 - accuracy: 0.9452 - val_loss: 0.3655 - val_accuracy: 0.8651
Epoch 4/10
218/218 [==============================] - 5s 23ms/step - loss: 0.0970 - accuracy: 0.9663 - val_loss: 0.4491 - val_accuracy: 0.8617
Epoch 5/10
218/218 [==============================] - 5s 23ms/step - loss: 0.0565 - accuracy: 0.9818 - val_loss: 0.5828 - val_accuracy: 0.8530
Epoch 6/10
218/218 [==============================] - 5s 24ms/step - loss: 0.0322 - accuracy: 0.9895 - val_loss: 0.6541 - val_accuracy: 0.8462
Epoch 7/10
218/218 [==============================] - 5s 24ms/step - loss: 0.0196 - accuracy: 0.9937 - val_loss: 0.7498 - val_accuracy: 0.8462

In [94]:
%%time
classes_x = (model.predict(X_test) > 0.5).astype("int32")
print("memoria:",psutil.Process().memory_info().rss / (1024 * 1024))

memoria: 1744.515625
CPU times: user 1.7 s, sys: 80 ms, total: 1.78 s
Wall time: 1.29 s


In [88]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, classes_x)

array([[2990,  459],
       [ 594, 2821]])

In [89]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,classes_x)

0.8465909090909091